# The Task

Based on a user query, return an existing question which most closely resembles the user's query

In [1]:
# import libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing

import spacy
from spacy.lang.en import English
import spacy
EN = spacy.load('en_core_web_sm')

from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

## Import the Data

In [3]:
data = pd.read_csv('C:/Users/sai ganesh reddy/Desktop/preprocessed_data.csv')
data

,answers_content,original_title,overall_scores,post_corpus,question_content,question_url,sentiment_polarity,sentiment_subjectivity,tags,processed_title
0,So for:,Format timedelta to string,-0.086345,format timedelta string python newbie 2 weeks ...,format timedelta string python newbie 2 weeks ...,https://stackoverflow.com/questions/538666,0.000000,0.000000,python|string|datetime|format|timedelta,format timedelta string
1,"MySQLdb has a separate cursor class for this, ...",Python: use mysqldb to import a MySQL table as...,0.036069,python use mysqldb import mysql table dictiona...,python use mysqldb import mysql table dictiona...,https://stackoverflow.com/questions/2180226,0.000000,0.000000,python|mysql|dictionary,python use mysqldb import mysql table dictionary
2,The second slice slices the sequence returned ...,slicing behaviour question of a list of lists,-0.063931,slicing behaviour question list lists got func...,slicing behaviour question list lists got func...,https://stackoverflow.com/questions/2466941,0.125000,0.166667,python|function|slice,slicing behaviour question list lists
3,You can extend Django's serializers without /t...,Serialize the @property methods in a Python class,-0.060483,serialize property methods python class way pr...,serialize property methods python class way pr...,https://stackoverflow.com/questions/2587119,0.141667,0.700000,python|django|serialization,serialize property methods python class
4,"Felix already provided an excellent answer, bu...",How to clone or copy a list?,0.913655,clone copy list options clone copy list python...,clone copy list options clone copy list python,https://stackoverflow.com/questions/2612802,0.076273,0.618391,python|list|copy|clone,clone copy list
5,"When creating Python classes, they by default ...",How is __slots__ implemented in Python?,-0.051862,_ _ slots _ _ implemented python creating pyth...,_ _ slots _ _ implemented python,https://stackoverflow.com/questions/5057845,0.144792,0.787153,python|python-3.x|python-c-api|descriptor|slots,_ _ slots _ _ implemented python
6,NaN,Fastest way to get the first object from a que...,0.003310,fastest way get first object queryset django o...,fastest way get first object queryset django o...,https://stackoverflow.com/questions/5123839,0.000000,0.000000,python|django|performance|django-models,fastest way get first object queryset django
7,Try using import matplotlib.pyplot as plt\n\nf...,Improve subplot size/spacing with many subplot...,0.491241,improve subplot size spacing many subplots mat...,improve subplot size spacing many subplots mat...,https://stackoverflow.com/questions/6541123,-0.007937,0.357143,python|matplotlib,improve subplot size spacing many subplots mat...
8,I'm also coming from Ruby so I love the syntax...,Python: Assign Value if None Exists,-0.057034,python assign value none exists ror programmer...,python assign value none exists ror programmer...,https://stackoverflow.com/questions/7338501,0.166667,0.308974,python|python-2.7|variable-assignment|language...,python assign value none exists
9,"If you want to join a list of strings, you can...",printing a tuple with a tab between the values,-0.058759,printing tuple tab values text file like one w...,printing tuple tab values text file like one,https://stackoverflow.com/questions/7424325,0.000000,0.000000,python|file|text|formatting,printing tuple tab values


## Import saved WordEmbeddings

In [5]:
# Import saved Wordvec Embeddings
import gensim
w2v_model = gensim.models.word2vec.Word2Vec.load('C:/Users/sai ganesh reddy/Desktop/SO_word2vec_embeddings.bin')

## Calculate Sentence Embeddings
In order to calculate the embeddings for an entire sentence, I defined the following function which averages the the embeddings for each valid token 

In [6]:
def question_to_vec(question, embeddings, dim=300):
    question_embedding = np.zeros(dim)
    valid_words = 0
    for word in question.split(' '):
        if word in embeddings:
            valid_words += 1
            question_embedding += embeddings[word]
    if valid_words > 0:
        return question_embedding/valid_words
    else:
        return question_embedding

In [7]:
'''
all_title_embeddings = []
for title in data.processed_title:
    all_title_embeddings.append(question_to_vec(title, w2v_model))
all_title_embeddings = np.array(all_title_embeddings)

embeddings = pd.DataFrame(data = all_title_embeddings)
embeddings.to_csv('models/title_embeddings.csv', index=False)
'''

"\nall_title_embeddings = []\nfor title in data.processed_title:\n    all_title_embeddings.append(question_to_vec(title, w2v_model))\nall_title_embeddings = np.array(all_title_embeddings)\n\nembeddings = pd.DataFrame(data = all_title_embeddings)\nembeddings.to_csv('models/title_embeddings.csv', index=False)\n"

Since the number of titles have have is fixed, I saved the sentence embeddings for all titles in a .csv file to save computation time on future runs 

In [9]:
all_title_embeddings = pd.read_csv('C:/Users/sai ganesh reddy/Desktop/title_embeddings.csv').values

## Import the saved model

In [10]:
import keras.backend as K

# Custom loss function to handle multilabel classification task
def multitask_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

Using TensorFlow backend.


In [12]:
from keras.models import load_model
import keras.losses

keras.losses.multitask_loss = multitask_loss
model = load_model('C:/Users/sai ganesh reddy/Desktop/Tag_predictor.h5')

In [13]:
def predict_tags(text, include_neutral=True):
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_SEQUENCE_LENGTH)
    # Predict
    prediction = model.predict([x_test])[0]
    for i,value in enumerate(prediction):
        if value > 0.5:
            prediction[i] = 1
        else:
            prediction[i] = 0
    tags = tag_encoder.inverse_transform(np.array([prediction]))
    return tags

In [14]:
import re
import nltk
import inflect
from nltk.corpus import stopwords

def tokenize_text(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

def preprocess_text(text):
    return ' '.join(normalize(tokenize_text(text)))

## Import the saved Tokenizer

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_SEQUENCE_LENGTH = 300
import pickle
with open('C:/Users/sai ganesh reddy/Desktop/stackoverflow/models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

## Getting the most similar results
So the way we actually calculate the the most similar results, is by comparing how far each result is from the query in terms of distance. This can only be done if both the query and the results are in a shared vector space. **Fortunately, that is exactly what our word embeddings are for**. They create each sentence as a vector in the embedding space, which makes it easy for us to distinguish them. 

After we have those vectors, we can assign a **Similarity Measure** as a metric which measures the closeness of two vectors. Common examples are Cosine distance, Euclidean distance and more.

**However, for this specific task, I decided to assign a custom similarity measure**. It is defined as follows:

![Similarity Measure](jupyter_imgs/similaritymeasure.png)

- It considers the cosine distance as a base measure
- It takes into account the popularity of the post based on the votes it has received by users at StackOverflow
- It takes into account the overall sentiment of the responses that people have made. A positive sentiment entails that the answers were helpful and thus is a good post 

In [18]:
from IPython.display import HTML
import logging
from sklearn.metrics.pairwise import cosine_similarity

search_string = "Combine lists of lists" 
search_string = ' '.join(normalize(tokenize_text(search_string)))
results_returned = "5" 
search_vect = np.array([question_to_vec(search_string, w2v_model)])    # Vectorize the user query

# Calculate Cosine similarites for the query and all titles
cosine_similarities = pd.Series(cosine_similarity(search_vect, all_title_embeddings)[0])

# Custom Similarity Measure
cosine_similarities = cosine_similarities*(1 + 0.4*data.overall_scores + 0.1*(data.sentiment_polarity))

output =""
for i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
    output += '<a target="_blank" href='+ str(data.question_url[i])+'><h2>' + data.original_title[i] + '</h2></a>'
    output += '<h3> Similarity Score: ' + str(j) + '</h3>'
    output += '<h3> Stackover Votes: ' + str(data.overall_scores[i]) + '</h3>'
    output +='<p style="font-family:verdana; font-size:110%;"> '
    for i in data.question_content[i][:50].split():
        if i.lower() in search_string:
            output += " <b>"+str(i)+"</b>"
        else:
            output += " "+str(i)
    output += "</p><hr>"
    
output = '<h3>Results:</h3>'+output
display(HTML(output))

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
